<center><h1>Đồ án Cuối Kì - Ứng Dụng Lập Trình Song Song<h1></center>

## I. Thông tin thành viên nhóm

|  |  Nhóm |MSSV | Họ và Tên|
| --- | ---| -------- | ----------- |
| 1 | Nhóm 7|19127353 | Lê Tấn Đạt|
| 2 | Nhóm 7|19127593 | Lê Tiến Trí|

## II. Link Github

Link github của nhóm: https://github.com/trile101/LTSSUD-Group7

## III. Mô tả bài toán và tập dữ liệu.

## Mô tả bài toán:
- Phân loại chữ số viết tay là một bài toán thuộc mảng Computer Vision, bài toán này đã giới thiệu từ rất lâu và hiện nay có rất nhiều mô hình đã giải quyết hiệu quả đạt được độ chính xác rất cao. Trong đó, bộ dữ liệu MNIST là bộ dữ liệu được sử dụng nhiều nhất cho bài toán này.
      
- Nhóm quyết định giải quyết bài toán phân loại chữ số viết tay MNIST bằng Convolutional Neural Network, kết hợp với kĩ thuật lập trình song song để tăng tốc quá trình huấn luyện.

- Mô hình mà nhóm hướng tới để thực hiện đó là mô hình CNN, sẽ được trình bày rõ hơn ở phần sau.

### Dataset, input và output của bài toán
- Bộ dữ liệu MNIST(Modified National Institute of Standards and Technology) là một bộ dữ liệu lớn chứa các chữ số viết tay thường được dùng trong việc huấn luyện các hệ thống xử lí hình ảnh, bao gồm 2 tập con: training set gồm 60.000 ảnh các chữ số viết tay và test set gồm 10.000 ảnh các chữ số.
MNIST nổi tiếng vì tần suất mà bộ dữ liệu được sử dụng.
- Bộ dữ liệu bao gồm 70.000 hình ảnh của các chữ số viết tay với sự phân chia như sau:
    - 60.000 hình ảnh đào tạo (training)
    - 10.000 hình ảnh thử nghiệm (testing)
- Input của bài toán:
    - ![](https://i0.wp.com/nttuan8.com/wp-content/uploads/2019/04/5.png?w=308&ssl=1)
    - Là một ảnh xám 28x28 pixel gồm các chữ số đơn viết tay từ 0 đến 9.
- Output: phân loại chính xác con số được biểu thị bởi hình input.
- Ví dụ ở đây với input như trên thì ta kì vọng output sẽ là 5.

### Những gì đã hoàn thành được đối với cả nhân và cả nhóm

- **Lê Tấn Đạt: 19127353**:
    - Implement phiên bản tuần tự của thuật toán CNN cho tập dữ liệu MNIST
    - Nghiên cứu các giải pháp song song hoá thuật toán tuần tự
- **Lê Tiến Trí: 19127593**:
    - Implement phiên bản song song của thuật toán CNN cho tập dữ liệu MNIST
    - Nghiên cứu các giải pháp song song hoá thuật toán tuần tự
- Cả nhóm:
    - Đã hoàn thành phiên bản tuần tự của thuật toán CNN.
    - Đã hoàn thành phiên bản song song hoá của thuật toán CNN.
    - Có 3 layer cần song song hoá: Convolutional Layer, Maxpooling layer và Fully Connected Layer. Đã song song hoá Convolutional Layer, đối với Maxpooling layer và Fully Connected layer thì nhóm sử dụng decorator jit(parallel=True) và jit(nopython = true) để tăng tốc độ chạy của thuật toán song song.

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import numba
from numba import jit, prange, cuda
from numba import config
config.THREADING_LAYER = 'omp'
from keras.datasets import mnist
from keras.utils import np_utils
import matplotlib.pyplot as plt
import timeit
import time
import math
from sklearn.metrics import accuracy_score

## IV. Thuật toán tuần tự, code và đánh giá

- Mô hình chính của bài toán đó là mô hình CNN, cụ thể các lớp sẽ được implement trong mô hình bao gồm:
    - Convolution Layer
    - Maxpooling Layer
    - Flatten Layer
    - Fully Connected Layer
    - Relu Layer
    - Softmax Layer
- Cùng với đó là hàm tính độ lỗi đối với bài toán phân loại có nhiều hơn 2 lớp đó **categorical_crossentropy**

### Class Layer

- Base class để các layer khác kế thừa, khi cần train mô hình chỉ cần gọi hàm forward, backward.

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass

### CNN Layer

- Minh hoạ:

![](https://topdev.vn/blog/wp-content/uploads/2019/08/Convolution_schematic.gif)
- Giải thích về lớp **Convolutional**: Lớp này là lớp đầu tiên được sử dụng để trích xuất các đặc trưng từ các hình ảnh đầu vào. Trong lớp này, phép toán tích chập được thực hiện giữa hình ảnh đầu vào và một số lượng filter có kích thước cụ thể MxM. Ta trượt bộ lọc qua hình ảnh đầu vào, tính dot product giữa bộ lọc và các phần của hình ảnh đầu vào tương ứng với kích thước của bộ lọc (MxM).
- Input đầu vào: ảnh xám input có kích thước 28*28, filter có kích thước 3x3, depth là số lượng filter sẽ sử dụng. (28x28)
- Output: ảnh vuông mới có kích thước là (28-3+1 = 26). Do có 3 filter được sử dụng do đó kịch thước output sẽ là (26, 26 ,3)

Code

In [ ]:
class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        np.random.seed(0)
        self.kernels = np.random.randn(*self.kernels_shape)
        np.random.seed(0)
        self.biases = np.random.randn(*self.output_shape)


    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

### Maxpooling Layer

- Minh hoạ:

![](https://media.geeksforgeeks.org/wp-content/uploads/20190721025744/Screenshot-2019-07-21-at-2.57.13-AM.png)
- Giải thích về lớp **Maxpooling**:
    - Thao tác pooling bao gồm việc trượt một bộ lọc hai chiều trên mỗi kênh của input và tóm tắt các đặc trưng nằm trong vùng được bao phủ bởi bộ lọc.
    - Có hai thao tác pooling thường gặp: maxpooling (lấy max) và average pooling ( lấy giá trị trung bình).
    - Lớp pooling sẽ giảm bớt số lượng tham số khi hình ảnh quá lớn. Không gian pooling còn được gọi là lấy mẫu con hoặc lấy mẫu xuống làm giảm kích thước của mỗi map nhưng vẫn giữ lại thông tin quan trọng.
- Input đầu vào: ảnh xám input có kích thước 26*26, filter có kích thước 2x2, stride là 2.
- Output: ảnh vuông mới có kích thước là (13, 13)

In [ ]:
class MaxPoolingLayer(Layer):
  def __init__(self, input_shape, kernel_size, stride):

      input_depth, input_height, input_width = input_shape
      self.depth = input_depth
      self.stride = stride
      self.input_shape = input_shape
      self.output_shape = (self.depth, int((input_height - kernel_size) / stride) + 1, int((input_width - kernel_size) / stride) + 1)
      self.kernels_size = kernel_size

      self.prev_input = np.zeros(input_shape)


  def get_pools(self, input, depth):

      pools = []

      for i in np.arange(input.shape[0], step=self.stride):

          for j in np.arange(input.shape[1], step=self.stride):

              mat = input[i:i+self.kernels_size, j:j+self.kernels_size]

              if mat.shape == (self.kernels_size, self.kernels_size):

                 pools.append(np.max(mat))

                 idx = np.where(input[i:i+self.kernels_size,j:j+self.kernels_size] == np.max(mat)) # find index where max element store in.
                 self.prev_input[depth][i + idx[0][0],j + idx[1][0]] = input[i + idx[0][0],j + idx[1][0]] # store in prev_input.


      tgt_shape = (self.output_shape[1], self.output_shape[2])
      return np.array(pools).reshape(tgt_shape)

  def max_pooling(self, input):

      num_pools = input.shape[1]
      tgt_shape = (self.output_shape[1], self.output_shape[2])
      pooled = []

      for pool in input:
          pooled.append(np.max(pool))

      return np.array(pooled).reshape(tgt_shape)

  def forward(self, input):
    self.input = input
    self.output = np.zeros(self.output_shape)
    for i in range(self.depth):
        self.output[i] = self.get_pools(self.input[i], i)

    return self.output

  def backward(self, output_gradient, learning_rate):
      return self.prev_input

### Flatten Layer

- Output của CNN có dạng ma trận (2 chiều), ta cần phải "trải phẳng" ra trước khi đưa qua lớp Fully Connected. Do đó, ta cần một layer gọi là Reshape để thay đổi shape sang hình dạng 1 chiều.

In [ ]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

### Fully Connected Layer

- Minh hoạ:

![](https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/2_fully-connected-layer.jpg)
- Giải thích về lớp **Fully Connected**:
    - Tầng kết nối đầy đủ (FC) nhận đầu vào là các dữ liệu đã được làm phẳng, mà mỗi đầu vào đó được kết nối đến tất cả neuron.
    - Tầng này có chức năng chuyển ma trận đặc trưng ở tầng trước thành vector chứa xác suất của các đối tượng cần được dự đoán. Ví dụ, trong bài toán phân loại số viết tay MNIST có 10 lớp tương ứng 10 số từ 0-1, tầng fully connected layer sẽ chuyển ma trận đặc trưng của tầng trước thành vector có 10 chiều thể hiện xác suất của 10 lớp tương ứng.
    - Công thức tính cho lớp Fully Connected:

        - ![](https://camo.githubusercontent.com/ea0bc2fad6c8958479956280d8bace2b146f3ec4ce31e0de13b672b99629bce4/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f3637342f302a4563396669586b4e4f6f41355a3276352e706e67)
        - Trong đó ```b``` là bias, ```w``` là weights (trọng số được khởi tạo mỗi khi gọi đến Layer)

- Input đầu vào: ảnh xám input có kích thước Mx1,
- Output: ảnh vuông mới có kích thước tuỳ chỉnh.

In [ ]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

### Relu Layer

- Minh hoạ:
    ![](https://www.baeldung.com/wp-content/ql-cache/quicklatex.com-05dbf7c989503746df6a2204dbf5a36a_l3.svg)
- Giải thích về hàm **Relu**:
    - Trong mạng thần kinh, hàm kích hoạt chịu trách nhiệm chuyển đổi đầu vào có trọng số tổng từ nút thành kích hoạt nút hoặc đầu ra cho đầu vào đó.
    - Hàm kích hoạt tuyến tính được chỉnh lưu (Rectified Linear Unit) hay được gọi là ReLU là một hàm tuyến tính từng phần sẽ xuất đầu vào trực tiếp nếu nó dương, nếu không, nó sẽ xuất ra số 0.
    - Hầu hết người ta thường dùng ReLU cho mô hình CNN vì nó có hiệu suất tốt.

In [ ]:
class Relu(Layer):
    def __init__(self, alpha = 0.1):
        self.alpha = alpha

    def forward(self, inputs):
        self.inputs = inputs.copy()
        self.outputs = np.maximum(0, inputs)
        return self.outputs

    def backward(self, output_gradient, learning_rate):
        self.input_gradient = output_gradient.copy()
        self.input_gradient[self.inputs < 0] = 0
        return self.input_gradient

### Softmax layer


- Vì đây là bài toán phân loại với số lượng lớp lớn hơn 2 (10 lớp) do đó hàm Softmax sẽ được sử dụng ngay sau lớp Fully Connected để tính toán ra được xác xuất của từng lớp đối với input đầu vào.
- Hàm softmax có hai đặc tính:
    - Xác suất sẽ luôn nằm trong khoảng (0:1].
    - Tổng tất cả các xác suất bằng 1.
- Nguyên lý của hàm  Softmax đối với một vài số cho trước:
    - Tính hàm lũy thừa số e, với số mũ là những số đã cho
    - Tính tổng các lũy thừa đó. Đó sẽ là mẫu số.
    - Sử dụng lũy thừa của mỗi số là tử số
    - Xác suất sẽ là tử số/mẫu số
    - Công thức hàm Softmax:

        ![](https://lh3.googleusercontent.com/5WZx0XW3sYFI7iQ8qP09vrAatoBGXtkVLs8H40FhrrNhu0TKH-W7mcMWGaH84S5CcJfx_aS_7U49H_6Lvfu056iZH9fUtKvuFOliMx81egv2MnIEjm9zOsWS4VNx2e2MZoows17j)

In [ ]:
class Softmax(Layer):
    def forward(self, input):
        self.input = input
        tmp = np.exp(self.input)
        self.output = tmp / np.sum(tmp)
        return self.output

    def backward(self, output_gradient, learning_rate):
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)

### Loss function

In [ ]:
def categorical_crossentropy(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 10**-100))

def categorical_crossentropy_backpropagation(y_true, y_pred):
    return -y_true/(y_pred + 10**-100)

### Xây dựng Network

In [ ]:
def predict(network, input):
    output = input
    fc_time = 0
    for layer in network:
        start = time.time()
        output = layer.forward(output)
        end = time.time()
        layer_time = end - start
        if layer.__class__.__name__ == "Convolutional":
            conv_time = layer_time
        if layer.__class__.__name__ == "MaxPoolingLayer":
            pool_time = layer_time
        if layer.__class__.__name__ == "FCLayer":
            fc_time += layer_time
    return output, conv_time, pool_time, fc_time

def predict_test(network, x_test, y_test, verbose = True):
    error = 0
    y_pred = y_test[0]
    true_label = 0

    for x, y in zip(x_test, y_test):
        output = x
        for layer in network:
            output = layer.forward(output)
        y_pred = output

        if np.argmax(y_pred) == np.argmax(y):
            true_label += 1

        # error
        error += categorical_crossentropy(y, y_pred)

        # backward
        grad = categorical_crossentropy_backpropagation(y, y_pred)
        for layer in reversed(network):
            grad = layer.backward(grad, 0.1)

    if verbose:
        print(f"Accuracy Score = {true_label/y_test.shape[0]}")


def train(network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, verbose = True):

    for e in range(epochs):
        total_time_conv = 0
        total_time_pool = 0
        total_time_fc = 0
        start = time.time()
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output, ct, pt, fct = predict(network, x)
            total_time_conv += ct
            total_time_pool += pt
            total_time_fc += fct

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        end = time.time()
        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}, time={end-start}")
            print(f"Total Convolution Time={total_time_conv}")
            print(f"Total MaxPooling Time={total_time_pool}")
            print(f"Total FullyConnected Time={total_time_fc}")
            print("")


In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 5),
    Relu(),
    MaxPoolingLayer((5,26,26), 2, 2),
    Reshape((5, 13, 13), (5 * 13 * 13, 1)),
    FCLayer(5 * 13 * 13, 100),
    Softmax(),
    FCLayer(100, 10),
    Softmax()
]

In [ ]:
def preprocess_data(x, y,limit=60000):
    x = x.reshape(len(x), 1, 28, 28)[:limit]
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 10, 1)[:limit]
    return x/255.0, y

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train,30000)
x_test, y_test = preprocess_data(x_test, y_test,10000)

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((30000, 1, 28, 28), (30000, 10, 1), (10000, 1, 28, 28), (10000, 10, 1))

In [ ]:
np.seterr(divide = 'ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [ ]:
%%time

train(
    network,
    categorical_crossentropy,
    categorical_crossentropy_backpropagation,
    x_train,
    y_train,
    epochs=5,
    learning_rate=0.1
)

1/5, error=2.3322510242131202, time=863.6049616336823
Total Convolution Time=12.570622444152832
Total MaxPooling Time=765.5276916027069
Total FullyConnected Time=23.04584789276123

2/5, error=2.3311105205075466, time=844.9227902889252
Total Convolution Time=11.976377010345459
Total MaxPooling Time=739.7580983638763
Total FullyConnected Time=27.319246768951416

3/5, error=2.331050419926125, time=838.0970630645752
Total Convolution Time=11.870171070098877
Total MaxPooling Time=733.2512247562408
Total FullyConnected Time=27.475435495376587

4/5, error=2.33101046837945, time=837.993923664093
Total Convolution Time=11.92070460319519
Total MaxPooling Time=733.2336919307709
Total FullyConnected Time=27.340416431427002

5/5, error=2.330980728993, time=833.4096159934998
Total Convolution Time=11.762924432754517
Total MaxPooling Time=729.327478647232
Total FullyConnected Time=26.972058296203613

CPU times: user 1h 11min 29s, sys: 50min 36s, total: 2h 2min 5s
Wall time: 1h 10min 18s


In [ ]:
%%time

predict_test(
    network,
    x_test,
    y_test,
)

Accuracy Score = 0.816
CPU times: user 4min 44s, sys: 3min 20s, total: 8min 5s
Wall time: 4min 40s


## V. Cài đặt song song


### 1. Ý tưởng phát triển bài toán

Sau quá trình đánh giá và nghiên cứu, nhóm tiến hành thực hiện áp dụng thư viện `numba` để phát triển song song ở một số layer và áp dụng sức mạnh của numba để tăng tốc độ của một số layer trong CNN.

![picture](https://drive.google.com/uc?export=view&id=1KQRGSqkREcdAYSht-ZnQTYAkknoECTY0)

**Với mô hình mạng `Convolution Neural Network` kết hợp với numba của python**
- `Lớp CNN` - sử dụng `numba(cuda.jit)`: Để thực hiện quá trình chạy song song trên GPU để tăng tốc độ của lớp này
- `Lớp Relu`, `MaxPooling`, `Fully Connected`, `Softmax`, `Category Cross Entropy` - sử dụng `numba(nonpython=True)`: Các lớp này sẽ được làm lại để có kết hợp với numba để tăng tốc so với lớp sử dụng thư viện numpy ban đầu

In [ ]:
import numpy as np
import numba
from numba import jit, prange, cuda
from numba import config
config.THREADING_LAYER = 'omp'
from keras.datasets import mnist
from keras.utils import np_utils
import matplotlib.pyplot as plt
import timeit
import time

In [ ]:
def preprocess_data(x, y,limit=60000):
    x = x.reshape(len(x), 28, 28,1)[:limit]
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 10)
    return x/255.0, y

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train, y_train=preprocess_data(x_train, y_train, 30000)
x_test, y_test = preprocess_data(x_test, y_test, 10000)

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass

### CNN Layer

**Ý tưởng và thiết kế CNN:**
- Trong quá trình huấn luyện CNN, ta truyền vào mini-batch nhất định, nên ta sẽ chia `grid_size = (số lượng ảnh input, số channel input)` từng ứng -> Nhằm mục đích với mỗi block sẽ xử lí một ảnh chỉ có chứa 1 channel duy nhất.
- Với mỗi block thì sẽ cho kích thước tương ứng với mỗi giai đoạn cần xử lí:
  - Hàm `convolve_forward` sẽ có block_size là (output_height, output width).
    - Hàm `backward_input` sẽ có block_size là (input height, input width).
    - Hàm `backward_kernel` sẽ có block_size là (output_gradient height, output_gradient width)

#### a. Lan truyền tiến

- Với mỗi thread sẽ thực hiện nhiệm vụ của mỗi điểm ảnh output

![picture](https://drive.google.com/uc?export=view&id=1qcUABIwnXdO-uGKEZMOkGMsEhzGykugj)

In [ ]:
def convolve_forward_device(imgs, filter, bias, output):
  '''
  Function - Áp dụng tính tổng tích các điểm giữa 2 ma trận trên ảnh và filter với kích thước (filter.width, filter.height, filter.layer) để có thể tính ra điểm output trên mỗi thread
  '''
  tx = cuda.threadIdx.x
  ty = cuda.threadIdx.y
  ff = cuda.blockIdx.y
  ii = cuda.blockIdx.x

  for c_img in range(filter.shape[1]):
      for hh in range(filter.shape[2]):
        for ww in range(filter.shape[3]):
          imgval = imgs[ii, ty + hh, tx + ww, c_img]
          filterval = filter[ff,c_img, hh, ww]
          output[ii, ty, tx, ff] += imgval * filterval
  output[ii, ty, tx, ff] += bias[ty,tx,ff]

#### b. Lan truyền ngược

1iD5nQB2g3LlIbI6Pk9uQT5Bds3KgwOPM

- Quá trình lan truyền ngược: Sau khi nhận được gradient của lớp sau truyền vào thì lớp Convolution sẽ bao gồm 2 bước:
  - Bước 1: Lớp convolution sẽ sử dụng gradient được truyền vào để đi tính lại bộ trọng số của bộ lọc để giảm thiểu sai số
  - Bước 2: Sẽ đi tính gradient sau khi bộ trọng số được cập nhật để cho lớp trước đó sử dụng
- Dựa vào các bước đó, ta chia song song hóa của quá trình lan truyền ngược làm 2 phần
  - Phần 1 (backward_kernel_device): Ta sẽ dựa theo công thức để áp dụng cho từng điểm trên filter
  ![picture](https://drive.google.com/uc?export=view&id=1LrQgZpbltmqIEs8nN7LYha8h7wDykqgm)
  - Phần 2 (backward_input_device): Ta sẽ dựa theo công thức để áp dụng cho từng điểm trên filter
  ![picture](https://drive.google.com/uc?export=view&id=1iD5nQB2g3LlIbI6Pk9uQT5Bds3KgwOPM)

In [ ]:
@cuda.jit
def backward_kernel_device(input, output_gradient, output):
  '''
  Function - Tính đạo hàm của mỗi lớp filter theo hàm loss dựa trên input và gradient của lớp trước
  '''
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    ii = cuda.blockIdx.x
    ff = cuda.blockIdx.y
    n_imgs = input.shape[0]

    for c_img in range(input.shape[3]):
        for hh in range(output.shape[2]):
          for ww in range(output.shape[3]):
            temp = input[ii, ty + hh, tx + ww, c_img] * output_gradient[ii, ty, tx , ff] / n_imgs
            output[ff, c_img, hh, ww] += temp

@cuda.jit
def backward_input_device(output_gradient, kernel, output):
  '''
  Function - Tính đạo hàm của mỗi lớp input theo hàm loss dựa trên filter và gradient của lớp trước
  '''
  tx = cuda.threadIdx.x
  ty = cuda.threadIdx.y
  ii = cuda.blockIdx.x
  c_img = cuda.blockIdx.y

  for ff in range(kernel.shape[0]):
    for hh in range(kernel.shape[2]):
      for ww in range(kernel.shape[3]):
          temp = output_gradient[ii, ty + hh, tx + ww, ff] * kernel[ff, c_img, hh, ww]
          output[ii, ty, tx, c_img] += temp

#### C. Class - Convolution

In [ ]:
class CNNLayer_Device(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape

        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth

        self.output_shape = (input_height - kernel_size + 1, input_width - kernel_size + 1,depth)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        np.random.seed(10)
        self.kernels = np.random.randn(*self.kernels_shape)
        np.random.seed(10)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        # Phân bố số lượng kích thước của grid và block
        self.input = input
        self.output = np.zeros((input.shape[0], self.output_shape[0], self.output_shape[1], self.output_shape[2]))
        grid_size = (input.shape[0], self.depth)
        block_size = (self.output.shape[1], self.output.shape[2])
        # Copy dữ liệu từ CPU sang GPU
        d_in = cuda.to_device(input)
        d_b = cuda.to_device(self.biases)
        d_k = cuda.to_device(self.kernels)
        d_out = cuda.to_device(self.output)
        # Thực hiện chạy function để song song hóa với grid size và block size được thiết lập
        convolve_forward_device[grid_size, block_size](d_in, d_k, d_b, d_out)
        cuda.synchronize()
        # Copy dữ liệu output từ GPU sang CPU
        self.output = d_out.copy_to_host()
        return self.output

    def backward(self, output_gradient, learning_rate):
        # Step_1: Tính đạo hàm filter
        kernel_size=self.kernels_shape[2]
        kernels_rot_180=np.rot90(self.kernels,2,axes=(2,3))
        kernels_rot_180 = np.ascontiguousarray(kernels_rot_180)

        d_k = cuda.to_device(kernels_rot_180)
        grid_size = (self.input.shape[0], self.depth)
        block_size = (output_gradient.shape[1], output_gradient.shape[2])
        kernels_gradient= np.zeros(self.kernels_shape)
        d_k_grad = cuda.to_device(kernels_gradient)
        d_in = cuda.to_device(self.input)
        d_out_grad = cuda.to_device(output_gradient)
        backward_kernel_device[grid_size, block_size](d_in, d_out_grad, d_k_grad)
        cuda.synchronize()
        kernels_gradient = d_k_grad.copy_to_host()

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * np.mean(output_gradient,axis=0)

        # Step_2: Tính đạo hàm input
        output_gradient=np.pad(output_gradient,[(0,0),(kernel_size-1,kernel_size-1),(kernel_size-1,kernel_size-1),(0,0)])
        d_out_grad = cuda.to_device(output_gradient)
        backward_gradient= np.zeros(self.input.shape)
        d_in_grad = cuda.to_device(backward_gradient)
        grid_size = (self.input.shape[0], self.input_depth)
        block_size = (self.input_shape[1], self.input_shape[2])
        backward_input_device[grid_size, block_size](d_out_grad, d_k, d_in_grad)
        cuda.synchronize()

        backward_gradient = d_in_grad.copy_to_host()

        return backward_gradient

### Maxpooling Layer

- Với max pooling layer, ta sử dụng kết hợp với numba(@jit(nopython=True) để có thể tăng tốc tốc độ của lớp này bằng cách
  - Thay thể việc sử dụng numpy bằng cách sử dụng các dòng for để tăng tốc độ

In [ ]:
@jit(nopython=True)
def maxpooling_forward(input, kernel_size, stride):
  n_imgs, n_rows, n_cols, c_imgs = input.shape
  output_shape = (n_imgs, int((n_rows - kernel_size) /stride + 1),int((n_cols - kernel_size) / stride + 1), c_imgs)
  output = np.zeros(output_shape)
  # lưu vị trí max của từng pool trong prev_input.
  prev_input = np.zeros((output_shape[0],output_shape[1],output_shape[2],output_shape[3],2))

  for ii in range(n_imgs):
    for c_img in range(c_imgs):
      for rr in range(0,n_rows):
        for cc in range(0,n_cols):

          idx_r = int((rr - kernel_size) // stride + 1)
          idx_c = int((cc - kernel_size) // stride + 1)
          temp_in = input[ii, rr, cc, c_img]
          temp_out = output[ii, idx_r, idx_c, c_img]
          if (temp_in > temp_out):
            output[ii, idx_r, idx_c, c_img] = temp_in
            prev_input[ii,idx_r, idx_c, c_img, 0] = rr
            prev_input[ii, idx_r, idx_c, c_img, 1] = cc

  return output, prev_input

@jit(nopython=True)
def maxpooling_backward(input, prev_input, prev_shape):
  '''
  Input
  input: input đã pooling
  prev_input: chứa thông tin vị trí max trước khi pool
  prev_shape: kích thước ban đầu trước khi pooling
  '''
  n_imgs, n_rows, n_cols, c_imgs = input.shape
  output = np.zeros(prev_shape)

  for ii in range(n_imgs):
    for c_img in range(c_imgs):
      for rr in range(n_rows):
        for cc in range(n_cols):
          idx = int(prev_input[ii, rr,cc,c_img, 0])
          idy = int(prev_input[ii, rr, cc, c_img, 1])
          output[ii, idx, idy, c_img] = input[ii, rr, cc, c_img] #gán max lại vị trí ban đầu.

  return output

class MaxPoolingLayer(Layer):
  def __init__(self, kernel_size, stride):
      self.kernel_size = kernel_size
      self.stride = stride

  def forward(self, input):
    output, prev_input = maxpooling_forward(input, self.kernel_size, self.stride)

    self.prev_input=prev_input
    self.pre_shape=input.shape
    return output

  def backward(self,input, learning_rate):
    '''
    input: input đã pooling
    prev_input: chứa thông tin vị trí max trước khi pool
    prev_shape: kích thước ban đầu trước khi pooling
    '''
    return maxpooling_backward(input, self.prev_input, self.pre_shape)

### Reshape Layer

In [ ]:
class Reshape(Layer):
    def __init__(self):
      pass

    def forward(self, input):
        self.input_shape=input.shape
        output_shape=1
        for i in self.input_shape[1:]:
          output_shape*=i
        self.output_shape=(self.input_shape[0],output_shape)
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

### Fully Connected Layer

- Với Fully Connected layer, ta sử dụng kết hợp với numba(@jit(parallel=True) để có thể tăng tốc tốc độ của lớp này bằng cách
  - Kết hợp các vòng for + thư viện numpy

In [ ]:
@jit(parallel=True)
def FC_forward(input, weight,bias):
  n_imgs = input.shape[0]

  res=np.zeros((n_imgs,weight.shape[1]))
  for nn in prange(n_imgs):
      for oo in prange(weight.shape[1]):
          res[nn][oo]=bias[oo]
          for ii in range(weight.shape[0]):
            res[nn][oo]+=input[nn][ii]*weight[ii][oo]

  return res

In [ ]:
@jit(parallel=True)
def FC_backward(input, weight, bias, output_gradient, learning_rate):

  n_imgs = input.shape[0]
  input_T=input.T

  # ---------------------------------------------------
  weight_gradient = np.zeros(weight.shape)

  for ii in prange(input_T.shape[0]):
    for oo in prange(output_gradient.shape[1]):
      for nn in range(n_imgs):
        weight_gradient[ii][oo]+=input_T[ii][nn]*output_gradient[nn][oo]/n_imgs

  weight-=learning_rate*weight_gradient

  for nn in prange(n_imgs):
    bias-=learning_rate*output_gradient[nn]/n_imgs

  weight_T=weight.T
  input_gradient = np.zeros(input.shape)

  for nn in prange(output_gradient.shape[0]):
    for ii in prange(weight_T.shape[1]):
      for oo in range(weight_T.shape[0]):
        input_gradient[nn][ii]+=output_gradient[nn][oo]*weight_T[oo][ii]

  return input_gradient

In [ ]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
      self.output_size=output_size
      np.random.seed(10)
      self.weights = np.random.randn(input_size, output_size)/100
      np.random.seed(10)
      self.bias = np.random.randn(output_size)/100

    def forward(self, input):
      self.input = input
      return FC_forward(self.input, self.weights,self.bias)

    def backward(self, output_gradient, learning_rate):
      return FC_backward(self.input, self.weights, self.bias, output_gradient, learning_rate)

### Relu Layer

In [ ]:
class Relu(Layer):
  def __init__(self, alpha = 0.1):
    self.alpha = alpha

  def forward(self, inputs):
    self.inputs = inputs.copy()
    self.outputs = np.maximum(0, inputs)
    return self.outputs

  def backward(self, output_gradient, learning_rate):
    self.input_gradient = output_gradient.copy()
    self.input_gradient[self.inputs < 0] = 0
    return self.input_gradient

### Softmax Layer

In [ ]:
@jit(nopython=True)
def softmax_forward(input):
  n_imgs = input.shape[0]
  output = np.zeros(input.shape)
  for nn in range(n_imgs):
    tmp = np.exp(input[nn] - np.max(input[nn]))

    output[nn] = tmp / np.sum(tmp)

  return output

class Softmax(Layer):
    def forward(self, input):
      self.output = softmax_forward(input)
      return self.output

    def backward(self, output_gradient, learning_rate):
      return output_gradient

### Loss Function

In [ ]:
@jit(nopython=True)
def categorical_crossentropy(y_true, y_pred):

  n_imgs = y_true.shape[0]
  num_class=y_true.shape[1]
  res = np.zeros(y_true.shape)
  for nn in range(n_imgs):
    for cc in range(num_class):
      res[nn]+=-np.log(y_pred[nn][cc])*y_true[nn][cc]
  return res

@jit(nopython=True)
def categorical_crossentropy_backpropagation(y_true, y_pred):
    n_imgs = y_true.shape[0]
    size = y_true.shape[1]
    res = np.zeros(y_true.shape)
    for nn in range(n_imgs):
      for cc in range(size):
        if y_true[nn][cc]!=0:
          res[nn][cc]=y_pred[nn][cc]-1
        else:
          res[nn][cc]=y_pred[nn][cc]
    return res/n_imgs

### Xây dựng Network

In [ ]:
def predict(network, input):
    output = input
    fc_time = 0

    for layer in network:
        start = time.time()
        output = layer.forward(output)
        layer_time = time.time() - start
        if layer.__class__.__name__ == "CNNLayer_Device":
          conv_time = layer_time
        if layer.__class__.__name__ == "MaxPoolingLayer":
            pool_time = layer_time
        if layer.__class__.__name__ == "FCLayer":
            fc_time += layer_time

    return output, conv_time, pool_time, fc_time


def train(network, loss, loss_prime, x_train, y_train, epochs = 1000,batch_size=600,learning_rate = 0.01, verbose = True):
  n=x_train.shape[0]
  batch_count=n//batch_size
  batch_index=[]

  for i in range(batch_count):
    batch_index.append((i*batch_size,(i+1)*batch_size))
  if batch_count*batch_size !=n:
    batch_index.append((batch_count*batch_size,n))
  for e in range(epochs):
    start=time.time()
    total_time_conv = 0
    total_time_pool = 0
    total_time_fc = 0
    error = 0

    for i, j in batch_index:
      output, ct, pt, fct = predict(network, x_train[i:j])
      total_time_conv += ct
      total_time_pool += pt
      total_time_fc += fct
      grad = loss_prime(y_train[i:j], output)
      loss_batch=loss(y_train[i:j], output)
      error += np.sum(loss_batch)/len(x_train)
      for layer in reversed(network):
          grad = layer.backward(grad, learning_rate)

    if verbose:
      end=time.time()
      print(f"{e + 1}/{epochs}, error={error}, time={end-start}")
      print(f"Total Convolution Time={total_time_conv}")
      print(f"Total MaxPooling Time={total_time_pool}")
      print(f"Total FullyConnected Time={total_time_fc}")
      print("")

In [ ]:
%%time
network = [
    CNNLayer_Device((1, 28, 28), 3, 32),
    Relu(),
    MaxPoolingLayer(2, 2),
    Reshape(),
    FCLayer(32 * 13 * 13, 100),
    Softmax(),
    FCLayer(100, 10),
    Softmax()
]

train(
    network,
    categorical_crossentropy,
    categorical_crossentropy_backpropagation,
    x_train,
    y_train,
    epochs=5,
    batch_size=32,
    learning_rate=0.1
)

1/5, error=22.945242989479848, time=112.24739813804626
Total Convolution Time=6.153479814529419
Total MaxPooling Time=11.187533140182495
Total FullyConnected Time=23.235563278198242

2/5, error=22.3911656111799, time=111.19126558303833
Total Convolution Time=6.15462589263916
Total MaxPooling Time=11.031086444854736
Total FullyConnected Time=23.189550161361694

3/5, error=21.443877967818956, time=111.65131831169128
Total Convolution Time=6.076167106628418
Total MaxPooling Time=11.203714609146118
Total FullyConnected Time=23.353671073913574

4/5, error=20.513881781837583, time=112.0895082950592
Total Convolution Time=6.1742939949035645
Total MaxPooling Time=11.189940452575684
Total FullyConnected Time=23.58342671394348

5/5, error=19.702200111378566, time=109.01741003990173
Total Convolution Time=6.234785795211792
Total MaxPooling Time=11.163268566131592
Total FullyConnected Time=22.751375436782837

CPU times: user 14min 46s, sys: 16.1 s, total: 15min 2s
Wall time: 9min 16s


In [ ]:
%%time
y_pred = predict(network, x_test)[0]

CPU times: user 18.8 s, sys: 1.83 s, total: 20.7 s
Wall time: 14.3 s


In [ ]:
true_label = 0
for x, y in zip(y_pred, y_test):
   if np.argmax(x) == np.argmax(y):
     true_label += 1

In [ ]:
print(f"Score: {true_label/y_pred.shape[0]}")

Score: 0.8208


## VI. Đánh giá kết quả


### Layer trong Mạng CNN

| Layer | Tuần tự | @jit(nonpython=true) |@jit(parallel=true) | @cuda.jit |
| --- | --- | --- | --- | --- |
| Convolution Layer | 12.58s |  | | 6.1534s |
| Maxpooling Layer| 765.52s | 11.18s |  |  |
| Fully connected layer| 27.32s | | 23.235s |  |

- **Nhận xét:** Dựa trên bảng mô tả tốc độ chạy layer ở trên của tuần tự và song song, ta thấy được tốc độ các lớp ở tuần tự đã được cải thiện một cách đáng kể đặc biệt là 2 lớp Convolution Layer(Giảm gần 50%) và  MaxPooling Layer (Giảm 98%)

## VII. Kết luận

- Sau quá trình hoàn thành đồ án, nhóm đã học được nhiều kiến thức về lập trình song song và các áp dụng nó để có thể tối ưu các thuật toán tuần tự
- Đồng thời, chủ đồ nhóm chọn là mạng CNN nên nhóm đã học rất nhiều và kĩ lưỡng về từng lớp có trong CNN và đặc điểm của chúng.
- **Khó khăn:** nhóm vẫn chưa có thể áp dụng lập trình song song trên GPU ở trên các lớp như (lớp Maxpooling, lớp Fully Connected) nên vẫn chưa đưa ra được một mô hình hoàn hảo

## VIII. Tài liệu tham khảo

- https://github.com/pradeepsinngh/Parallel-Deep-Learning-in-Python
- https://deeplearning.cs.cmu.edu/F21/document/recitation/Recitation5/CNN_Backprop_Recitation_5_F21.pdf
- https://nttuan8.com/bai-6-convolutional-neural-network/
- https://www.kaggle.com/talevy23/cnn-with-parallel-convolution-layers
- http://cucis.ece.northwestern.edu/publications/pdf/LJA17.pdf